<a href="https://colab.research.google.com/github/akhavan12/discrete_covid_model/blob/master/Discrete_Model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
from ipywidgets import GridspecLayout
from ipywidgets import FloatSlider
pd.options.display.max_rows = 999
import scipy as scipy
from scipy.interpolate import make_interp_spline, BSpline

In [0]:
df_Italy = pd.read_excel('https://github.com/akhavan12/discrete_covid_model/raw/master/Country_Trends.xlsx',sheet_name='Italy')

# Algorithm


![alt text](https://raw.githubusercontent.com/akhavan12/discrete_covid_model/master/Model_Discrete_v8.svg)

In [0]:
## Each state represents time from onset in one conditions (E,Symp,M,W,V)
## when moving from one day to another, the states increment (E(t) -> E(t+1))
## the change or staying in each condition, and state has it's own probability

#H -- > E[ state 0, time t]--> P_E(state,E) --> + E(State 1, t+1)
#                          --> P_E(state,Symp) --> + Symp(State 0, t+1) -->(Testing ratio)
#                          --> P_H(state,H) --> + H(t+1)
#S(state=1,t) -- > M(state=0,t+1) --P_M(state=1~14,M)-- > M(state=1,t+2)
#                                 --P_M(state=1~14,W)-- > W(state=1,t+2)-----|P_W(state=1~14,V) ---> V(state=0, t=t+3) -->| P_R(state=1~14,V) --> R
#                                 --P_M(state=1~14,R)-- > R(state=1,t+2)     |P_W(state=1~14,D) --->                      | P_D(state=1~14,V) --> D
#                                                                            |P_R(state=1~14,R) ---> 
# E(state 0,  t+1) = H(t)* (M + W)*Beta/N
# H(t+1) = H(t) - E(State 0, t+1)

# Transition Matrices

In [0]:
def set_probs_M(Probs):
  ### M,R,Severe
  Probs["M"] = [
          [1.00,0.00,0.00], #0
          [0.80,0.10,0.10], #1
          [0.80,0.10,0.10], #2
          [0.70,0.20,0.10], #3
          [0.70,0.20,0.10], #4
          [0.70,0.20,0.10], #5
          [0.60,0.30,0.10], #6
          [0.50,0.40,0.10], #7
          [0.50,0.40,0.10], #8
          [0.00,1.00,0.00], #9
          [0.00,1.00,0.00], #10
          [0.00,1.00,0.00], #11
          [0.00,1.00,0.00], #12
          [0.00,1.00,0.00], #13
          [0.00,1.00,0.00]] #14 
  
def set_probs_Severe(Probs):
  ### Severe,R,V,D
  Probs["Severe"] = [
          [1.00,0.00,0.00,0.00], #0
          [0.70,0.17,0.10,0.03], #1
          [0.70,0.17,0.10,0.03], #2
          [0.60,0.27,0.10,0.03], #3
          [0.57,0.30,0.10,0.03], #4
          [0.47,0.40,0.10,0.03], #5
          [0.47,0.40,0.10,0.03], #6
          [0.30,0.57,0.10,0.03], #7
          [0.20,0.67,0.10,0.03], #8
          [0.10,0.60,0.27,0.03], #9
          [0.05,0.60,0.32,0.03], #10
          [0.0,1.00,0.0,0.00], #11
          [0.0,1.00,0.0,0.00], #12
          [0.0,1.00,0.0,0.00], #13
          [0.0,1.00,0.0,0.00]]   #14


def set_probs_ventilator(Probs):
  ### V,R,D 
  Probs["ventilator"] = [
          [0.50,0.45,0.05], #0
          [0.40,0.55,0.05], #1
          [0.40,0.55,0.05], #2
          [0.30,0.65,0.05], #3
          [0.30,0.65,0.05], #4
          [0.20,0.75,0.05], #5
          [0.20,0.75,0.05], #6
          [0.20,0.75,0.05], #7
          [0.20,0.75,0.05], #8
          [0.20,0.75,0.05], #9
          [0.20,0.75,0.05], #10
          [0.20,0.75,0.05], #11
          [0.20,0.75,0.05], #12
          [0.20,0.75,0.05], #13
          [0.00,0.95,0.05]] #14



def set_probs_Symp(Probs):
  ### Symp,M,H.   0: not tested, 1: tested positive ,2: tested negative
  Probs['Symp'] =[[0.20,  0.80*0.70,  0.80*0.30], #0 Testing and + cases go to Mild cases , negative cases go back to Healthy
                  [0.00,  1.00,       0.00]]      #1 ###  all remainings go to Mild cases 

def set_probs_E(Probs):
  ### E,H,Symp
  Probs["E"] = [[1.00,0.00,0.00], #0 ### first day of exposure
                [0.30,0.50,0.20], #1        
                [0.20,0.50,0.30], #2 
                [0.10,0.60,0.40], #3
                [0.05,0.65,0.50], #4
                [0.00,0.60,0.60]] #5 



# Initialization

In [0]:
Probs={}
H = []
E = []
Symp = []
Tested = []
M = []
V = []
W = []
R = []
D = []
N = 0
Mild_days  =[]
Severe_days = []
Ventilator_days =[] 

def initialize(MaxTime, H_0, E_0_0, M_0_0, Symp_0_0):
  global H
  global E
  global Symp
  global Tested
  global M
  global V
  global W
  global R
  global D
  global Probs
  global N
  N = H_0

  H=np.zeros(MaxTime) ## healthy population has 1 class
  E=np.zeros([5,MaxTime]) ## Exposed population has 5 classes

  Symp=np.zeros([2,MaxTime]) ## Sympt population has 2 classes
  Tested = np.zeros(MaxTime) ## Sympt population has 2 classes
  M=np.zeros([15,MaxTime]) ## Mild population has 14 classes
  V=np.zeros([15,MaxTime]) ## Ventilator population has 14 classes
  W=np.zeros([15,MaxTime]) ## Severe population has 14 classes
  R=np.zeros(MaxTime) ## Recovered population has 1 class
  D=np.zeros(MaxTime) ## Dead population has 1 class
  
  H[0] = H_0
  E[0,0] = E_0_0
  M[0,0]=M_0_0
  Symp[0,0]=Symp_0_0



# Simulation function

In [0]:
def Simulate(MaxTime,Beta):
  global Mild_days
  global Severe_days
  global Ventilator_days
  global H
  global E
  global Symp
  global Tested
  global M
  global V
  global W
  global R
  global D
  global Probs

  results = []
  for t in range(MaxTime-1):
  #  Mild_days = Mild_days + M[:,t]
   # Severe_days = Severe_days + W[:,t]
   # Ventilator_days = Ventilator_days + V[:,t]
    ### transfer previous days number of recovered cases
    R[t+1] = R[t]
    D[t+1] = D[t]
   ### when the case has symptoms, it is removed from the population
    H[t+1] = H[t]- E[0,t]
  ### at time t calculate all the values of t+1 for all the states
    for state in range(0,4):
      ## Stay exposed conditions
      #print('t:',t,'-->',E[state,t]*Probs["E"][state][0],E[state,t]*Probs["E"][state][1],E[state,t]*Probs["E"][state][2],'-->',E[state,t])
      E[state+1,t+1] =  E[state,t]*Probs["E"][state][0] 
      ## go back to healthy population
      #H[t+1] = H[t+1] + E[state,t] * Probs["E"][state][1]
      ## become symtomic
      Symp[0,t+1] = Symp[0,t+1] + E[state,t] * Probs["E"][state][2]

    ### Symptomatic states 
    #################################################
    Symp[1,t+1] = Symp[0,t] * Probs["Symp"][0][0]         ### state 0 remain sympt
    Tested[t] = Tested[t] + Symp[0,t] * (1-Probs["Symp"][0][0]) 

    ### from Symptomatic to mild --> where testing also happens
    M[0,t+1] = Symp[0,t] * Probs["Symp"][0][1]           ### From state 0 to Mild state 0
    
    H[t+1] = H[t+1] + Symp[0,t] * Probs["Symp"][0][2]           ### From state 0 to Mild state 0

    M[0,t+1] = M[0,t+1] +  Symp[1,t] * Probs["Symp"][1][1] ### From state 1 to Mild state 0
    #################################################

    for state in range(0,14):
      ### M,R,Severe
      M[state+1,t+1] =  M[state,t] * Probs["M"][state][0] ## from state= state --> to state+1

      R[t+1] = R[t+1] + M[state,t] * Probs["M"][state][1] ## from Mild to recovered

      W[0,t+1] = W[0,t+1] + M[state,t] * Probs["M"][state][2]

    temp=0
    for state in range(0,14):
      ### Severe,R,V,D
      W[state+1,t+1] =  M[state,t] * Probs["Severe"][state][0]
      R[t+1] = R[t+1] + W[state,t] * Probs["Severe"][state][1]
      V[0,t+1] = V[0,t+1] +  W[state,t] * Probs["Severe"][state][2]
      temp = temp + W[state,t] * Probs["Severe"][state][2]
      
      D[t+1] = D[t+1] + W[state,t] * Probs["Severe"][state][3]
    for state in range(0,14):
      ### V,R,D 
      V[state+1,t+1] =  V[state,t] * Probs["ventilator"][state][0]
      R[t+1] = R[t+1] + V[state,t] * Probs["Severe"][state][1]

      D[t+1] = D[t+1] + V[state,t] * Probs["Severe"][state][2]


    results.append({
    "time":t,
    "Sucseptible" : H[t],
    "Mild_0": M[0,t].sum(),
    "Mild_1": M[1,t].sum(),
    "Mild_2": M[2,t].sum(),
    "Mild_3": M[3,t].sum(),
    "Mild_4": M[4,t].sum(),
    "Mild_5": M[5,t].sum(),
    "Mild_6": M[6,t].sum(),
    "Mild_7": M[7,t].sum(),
    "Mild_12": M[12,t].sum(),
    "E": E[:,t].sum(),
    "Symp_0" : Symp[0,t].sum(),
    "Symp_1" : Symp[1,t].sum(),
    "Symp" : Symp[:,t].sum(),
    "Mild" : M[:,t].sum(),
    "Severe" :  W[:,t].sum(),
    "Ventilator" : V[:,t].sum(),
    "Death" : D[t],
    "Recovered" : R[t],
    "Tested" : Tested[t]})
    
    E[0,t+1] = ( H[t] * (M[:,t].sum()+W[:,t].sum())/N)* Beta

  return pd.DataFrame(results)


# Static control of the results

In [0]:
H_0 = 6000000
E_0 = 800
B = 1.5
set_probs_M(Probs)
set_probs_Severe(Probs)
set_probs_Symp(Probs)
set_probs_ventilator(Probs)
set_probs_E(Probs)
MaxTime =90
initialize(MaxTime=MaxTime,H_0=H_0, E_0_0=E_0, M_0_0=0, Symp_0_0=0)
df_res=Simulate(MaxTime=MaxTime, Beta = B)


In [45]:
df_res['sums']=df_res[[	'Mild'	,'Severe',	'Ventilator',	'Death'	,'Recovered']].sum(axis=1)
disp=df_res[['Sucseptible','E','Symp',	'Mild'
        ,'Severe',	'Ventilator',	'Death'	
        ,'Recovered','sums']].tail(60).style.background_gradient(cmap='viridis')
disp

,Sucseptible,E,Symp,Mild,Severe,Ventilator,Death,Recovered,sums
29,5940068.725423,22411.169718,2691.031682,4897.547987,3407.994383,439.097329,649.760208,7405.741509,16800.141416
30,5930044.048368,26136.219838,3133.106664,5711.620344,3978.639311,512.806303,760.344647,8667.969950,19631.380556
31,5918351.902512,30463.126308,3652.540248,6656.471467,4639.057880,598.782562,889.425305,10141.353341,22925.090555
32,5904734.250071,35467.746506,4259.716900,7757.299984,5405.655107,698.538590,1039.959127,11859.767025,26761.219833
33,5888894.199884,41256.929336,4964.283039,9041.977726,6299.293387,814.284217,1215.416811,13862.916909,31233.889050
34,5870480.154030,47966.495617,5778.901406,10539.295829,7342.483904,948.975379,1419.913317,16197.815571,36448.484000
35,5849077.931331,55747.679525,6721.413500,12280.123329,8558.108686,1106.043879,1658.278783,18919.642204,42522.196881
36,5824210.700727,64756.635850,7814.069091,14300.377362,9970.774205,1289.141083,1936.099353,22092.276326,49588.668329
37,5795339.681026,75156.015863,9081.203627,16643.416846,11609.496127,1502.157837,2259.787558,25789.137043,57803.995411
38,5761859.076518,87125.349488,10547.769012,19360.431919,13509.562451,1749.504477,2636.716951,30094.714913,67350.930710


# Interactive form of the algorithm

In [0]:
df_results=pd.DataFrame()
def Interactive_deriv(H_0,E_0,B):

  gap=15


  global df_results
  set_probs_M(Probs)
  set_probs_Severe(Probs)
  set_probs_Symp(Probs)
  set_probs_ventilator(Probs)
  set_probs_E(Probs)
  initialize(MaxTime=100,H_0=H_0, E_0_0=E_0, M_0_0=10, Symp_0_0=10)
  results=Simulate(MaxTime=100, Beta = B)
  
  f,ax = plt.subplots(1,1,figsize=(10,8))
  #ax.plot(M.sum(axis=0),color='orange',label='mild')
  #ax.plot(W.sum(axis=0),color='navy',label='severe')
  #ax.plot(V.sum(axis=0),color='purple',label='ventilator')
  ax.plot(M.sum(axis=0)+W.sum(axis=0)+V.sum(axis=0),color='orange',label='simulation active cases')
  ax.plot(df_Italy.index+gap, df_Italy['active'],'--',color='orange',label='Italy active cases')
  #ax.plot(Tested.cumsum(),'-o',color='gray',label='Simulation')


  #ax.plot(R,color='green',label='recovered')
  ax.plot(D,color='red',label='dead')
  ax.plot(df_Italy.index+gap, df_Italy['total death'],'--',color='red',label='Italy total death')
  #ax.plot(H,color='black',label='healthy')
  ax.set_xlim(xmin=0,xmax=80)
  
  
  ax.plot(V.sum(axis=0) ,color='purple',label='ventilator')
  ax.plot(df_Italy.index+gap, df_Italy['icu'],'--',color='purple',label='active icu')
  #ax.plot(df_Italy.index+30, df_Italy['total test'],'-*',color='gray',label='Italy Total Tested')

  ax.legend(fontsize=14)
  plt.show()
  df_results=pd.DataFrame(results)

  return 

w = interactive(
    Interactive_deriv,
    H_0=      FloatSlider(min=10000, max=5000000, continuous_update=False, readout_format='.3f',   step=1000, value=3100000, description = 'H t0'), 
    E_0=      FloatSlider(min=10, max=50000,       continuous_update=False, readout_format='.3f',   step=1, value=10000, description = 'Exposed t0') ,
    B=        FloatSlider(min=0.50, max=3.00,     continuous_update=False, readout_format='.3f',   step=.05, value=.9, description = 'Beta'))


In [47]:
w

interactive(children=(FloatSlider(value=3100000.0, continuous_update=False, description='H t0', max=5000000.0,…